In [1]:
%matplotlib inline
import numpy as np
import scipy as sp
import matplotlib as mpl
import matplotlib.cm as cm
import matplotlib.pyplot as plt
import pandas as pd
pd.set_option('display.width', 500)
pd.set_option('display.max_columns', 100)
pd.set_option('display.notebook_repr_html', True)
import seaborn as sns
sns.set_style("whitegrid")
sns.set_context("poster")
import re
import csv
from sklearn.preprocessing import LabelBinarizer
from sklearn.preprocessing import LabelEncoder

/Users/hechengwang/anaconda/lib/python2.7/site-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')


In [2]:
train = pd.read_csv("train.csv", header = 0)
test = pd.read_csv("test.csv", header = 0)

In [3]:
def clean_data(df):
    
    #gender and embarked binarization
    #df = pd.concat([df, pd.get_dummies(df['Sex'])], axis = 1)
    #df = pd.concat([df, pd.get_dummies(df['Embarked'].fillna('S'), prefix= 'Embarked')], axis = 1)
    df['Embarked'] = df['Embarked'].fillna('S')  
    
    #create new feature called famsize
    df['FamilySize'] = df['SibSp'] + df['Parch'] + 1.0
    df['Family']=df['SibSp']*df['Parch']
    #fare fix
    df.loc[ (df.Fare.isnull())&(df.Pclass==1),'Fare'] =np.median(df[df['Pclass'] == 1]['Fare'].dropna())
    df.loc[ (df.Fare.isnull())&(df.Pclass==2),'Fare'] =np.median( df[df['Pclass'] == 2]['Fare'].dropna())
    df.loc[ (df.Fare.isnull())&(df.Pclass==3),'Fare'] = np.median(df[df['Pclass'] == 3]['Fare'].dropna())
    df['Fare_Per_Person'] = df['Fare'] / df['FamilySize']

    #title
    df['Title'] = df['Name'].str.extract("([\w]+\.)")
    def replace_titles(x):
        title=x['Title']
        if title in ['Mr.','Don.', 'Major.', 'Capt.', 'Jonkheer.', 'Rev.', 'Col.']:
            return 'Mr.'
        elif title in ['Master.']:
            return 'Master.'
        elif title in ['Countess.', 'Mme.','Mrs.']:
            return 'Mrs.'
        elif title in ['Mlle.', 'Ms.','Miss.']:
            return 'Miss.'
        elif title =='Dr.':
            if x['Sex']=='Male':
                return 'Mr.'
            else:
                return 'Mrs.'
        elif title =='':
            if x['Sex']=='Male':
                return 'Master.'
            else:
                return 'Miss.'
        else:
            return title

    df['Title']=df.apply(replace_titles, axis=1)
    #df = pd.concat([df, pd.get_dummies(df['Title'], prefix = 'Title')], axis = 1)
    
    #age fix
    df['AgeFill'] = df['Age'] \
        .groupby([df['Sex'], df['Title']]) \
        .apply(lambda x: x.fillna(x.mean()))
    
    df['AgeCat']=df['AgeFill']
    df.loc[ (df.AgeFill<=10) ,'AgeCat'] = 'child'
    df.loc[ (df.AgeFill>60),'AgeCat'] = 'aged'
    df.loc[ (df.AgeFill>10) & (df.AgeFill <=30) ,'AgeCat'] = 'adult'
    df.loc[ (df.AgeFill>30) & (df.AgeFill <=60) ,'AgeCat'] = 'senior'
    #df = pd.concat([df, pd.get_dummies(df['AgeCat'], prefix = 5)], axis = 1)
    
    #cabin fix
    df['Cabin_Category'] = df['Cabin'].str.extract("(\w)").fillna("")
    #df['Cabin_Category'] = np.where(df['Cabin_Category'] == "", 0, 1)
    
    
    #added features
    df['AgeClass']=df['AgeFill']*df['Pclass']
    df['ClassFare']=df['Pclass']*df['Fare_Per_Person']
    df['HighLow']=df['Pclass']
    df.loc[ (df.Fare_Per_Person<8) ,'HighLow'] = 0
    df.loc[ (df.Fare_Per_Person>=8) ,'HighLow'] = 1
    
    column_list = ['Sex', 'Embarked', 'Title', 'AgeCat', 'Cabin_Category']
    for column in column_list:
        le = LabelEncoder()
        le.fit(df[column])
        df[column] = le.transform(df[column])
    
    df = df.drop(['PassengerId', 'Age', 'Name', 'Cabin', 'Ticket'], axis = 1)
    return df

In [4]:
x = clean_data(train).drop('Survived', axis = 1)
y = train['Survived']
test_new = clean_data(test)

In [5]:
x

,Pclass,Sex,SibSp,Parch,Fare,Embarked,FamilySize,Family,Fare_Per_Person,Title,AgeFill,AgeCat,Cabin_Category,AgeClass,ClassFare,HighLow
0,3,1,1,0,7.2500,2,2,0,3.625000,3,22.000000,0,0,66.000000,10.875000,0
1,1,0,1,0,71.2833,0,2,0,35.641650,4,38.000000,3,3,38.000000,35.641650,1
2,3,0,0,0,7.9250,2,1,0,7.925000,2,26.000000,0,0,78.000000,23.775000,0
3,1,0,1,0,53.1000,2,2,0,26.550000,4,35.000000,3,3,35.000000,26.550000,1
4,3,1,0,0,8.0500,2,1,0,8.050000,3,35.000000,3,0,105.000000,24.150000,1
5,3,1,0,0,8.4583,1,1,0,8.458300,3,32.852798,3,0,98.558394,25.374900,1
6,1,1,0,0,51.8625,2,1,0,51.862500,3,54.000000,3,5,54.000000,51.862500,1
7,3,1,3,1,21.0750,2,5,3,4.215000,1,2.000000,2,0,6.000000,12.645000,0
8,3,0,0,2,11.1333,2,3,0,3.711100,4,27.000000,0,0,81.000000,11.133300,0
9,2,0,1,0,30.0708,0,2,0,15.035400,4,14.000000,0,0,28.000000,30.070800,1


In [6]:
from sklearn.ensemble import RandomForestClassifier
from sklearn import cross_validation
from sklearn import preprocessing
from sklearn.cross_validation import KFold
from sklearn.metrics import accuracy_score
import scipy
import matplotlib as mpl
from sklearn.linear_model import LogisticRegression
from sklearn.grid_search import GridSearchCV
from xgboost import XGBClassifier
from sklearn.metrics import confusion_matrix

In [7]:
x_train, x_test, y_train, y_test = cross_validation.train_test_split(x, y, test_size = .33)

In [8]:
clfl2=XGBClassifier()
parameters = {'n_estimators': [100, 200, 300, 400, 500], 
              'max_depth': range(1, 6), 
              'reg_lambda': [1],
              'reg_alpha': [1]}
fitmodel = GridSearchCV(clfl2, param_grid=parameters, cv=10, scoring="accuracy")
fitmodel.fit(x_train, y_train)
fitmodel.best_estimator_, fitmodel.best_params_, fitmodel.best_score_, fitmodel.grid_scores_

(XGBClassifier(base_score=0.5, colsample_bylevel=1, colsample_bytree=1,
        gamma=0, learning_rate=0.1, max_delta_step=0, max_depth=5,
        min_child_weight=1, missing=None, n_estimators=100, nthread=-1,
        objective='binary:logistic', reg_alpha=1, reg_lambda=1,
        scale_pos_weight=1, seed=0, silent=True, subsample=1),
 {'max_depth': 5, 'n_estimators': 100, 'reg_alpha': 1, 'reg_lambda': 1},
 0.8523489932885906,
 [mean: 0.82047, std: 0.02543, params: {'n_estimators': 100, 'reg_lambda': 1, 'reg_alpha': 1, 'max_depth': 1},
  mean: 0.82550, std: 0.03310, params: {'n_estimators': 200, 'reg_lambda': 1, 'reg_alpha': 1, 'max_depth': 1},
  mean: 0.83054, std: 0.03316, params: {'n_estimators': 300, 'reg_lambda': 1, 'reg_alpha': 1, 'max_depth': 1},
  mean: 0.83893, std: 0.03390, params: {'n_estimators': 400, 'reg_lambda': 1, 'reg_alpha': 1, 'max_depth': 1},
  mean: 0.84228, std: 0.03232, params: {'n_estimators': 500, 'reg_lambda': 1, 'reg_alpha': 1, 'max_depth': 1},
  mean: 0.828

In [9]:
clfl2=RandomForestClassifier()
parameters = {'n_estimators': [100, 200, 300, 400, 500], 
              'max_depth': range(1, 6)}
fitmodel = GridSearchCV(clfl2, param_grid=parameters, cv=10, scoring="accuracy")
fitmodel.fit(x_train, y_train)
fitmodel.best_estimator_, fitmodel.best_params_, fitmodel.best_score_, fitmodel.grid_scores_

(RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
             max_depth=5, max_features='auto', max_leaf_nodes=None,
             min_samples_leaf=1, min_samples_split=2,
             min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=1,
             oob_score=False, random_state=None, verbose=0,
             warm_start=False),
 {'max_depth': 5, 'n_estimators': 100},
 0.84899328859060408,
 [mean: 0.77852, std: 0.03600, params: {'n_estimators': 100, 'max_depth': 1},
  mean: 0.78691, std: 0.05154, params: {'n_estimators': 200, 'max_depth': 1},
  mean: 0.77852, std: 0.04718, params: {'n_estimators': 300, 'max_depth': 1},
  mean: 0.77349, std: 0.04698, params: {'n_estimators': 400, 'max_depth': 1},
  mean: 0.78020, std: 0.04511, params: {'n_estimators': 500, 'max_depth': 1},
  mean: 0.80537, std: 0.05444, params: {'n_estimators': 100, 'max_depth': 2},
  mean: 0.80537, std: 0.05158, params: {'n_estimators': 200, 'max_depth': 2},
  mean: 0.81376, std: 0.05714,

In [12]:
for i in range(1, 11):
    clf = RandomForestClassifier(n_estimators = 500, max_depth = i)
    clf.fit(x_train, y_train)
    x_train_pred = clf.predict(x_train)
    x_test_pred = clf.predict(x_test)
    print "train-test accuracy: %s - %s %s" %(accuracy_score(x_train_pred, y_train), accuracy_score(x_test_pred, y_test), i)

train-test accuracy: 0.771812080537 - 0.735593220339 1
train-test accuracy: 0.822147651007 - 0.779661016949 2
train-test accuracy: 0.857382550336 - 0.806779661017 3
train-test accuracy: 0.895973154362 - 0.810169491525 4
train-test accuracy: 0.904362416107 - 0.810169491525 5
train-test accuracy: 0.924496644295 - 0.806779661017 6
train-test accuracy: 0.937919463087 - 0.803389830508 7
train-test accuracy: 0.953020134228 - 0.793220338983 8
train-test accuracy: 0.961409395973 - 0.796610169492 9
train-test accuracy: 0.974832214765 - 0.796610169492 10


In [101]:
for i in range(1, 20):
    clf = RandomForestClassifier(n_estimators = 500, max_depth = 7)
    clf.fit(x_train, y_train)
    x_train_pred = clf.predict(x_train)
    x_test_pred = clf.predict(x_test)
    print "train-test accuracy: %s - %s %s" %(accuracy_score(x_train_pred, y_train), accuracy_score(x_test_pred, y_test), i)

train-test accuracy: 0.887583892617 - 0.837288135593 1
train-test accuracy: 0.88255033557 - 0.837288135593 2
train-test accuracy: 0.880872483221 - 0.833898305085 3
train-test accuracy: 0.887583892617 - 0.830508474576 4
train-test accuracy: 0.885906040268 - 0.823728813559 5
train-test accuracy: 0.887583892617 - 0.830508474576 6
train-test accuracy: 0.879194630872 - 0.820338983051 7
train-test accuracy: 0.889261744966 - 0.827118644068 8
train-test accuracy: 0.892617449664 - 0.833898305085 9
train-test accuracy: 0.889261744966 - 0.823728813559 10
train-test accuracy: 0.879194630872 - 0.820338983051 11
train-test accuracy: 0.887583892617 - 0.830508474576 12
train-test accuracy: 0.879194630872 - 0.833898305085 13
train-test accuracy: 0.877516778523 - 0.830508474576 14
train-test accuracy: 0.894295302013 - 0.837288135593 15
train-test accuracy: 0.875838926174 - 0.823728813559 16
train-test accuracy: 0.874161073826 - 0.827118644068 17
train-test accuracy: 0.884228187919 - 0.833898305085 18
tr

In [100]:
test_new.describe()

,Pclass,SibSp,Parch,Ticket,Fare,female,male,Embarked_C,Embarked_Q,Embarked_S,FamilySize,Family,Fare_Per_Person,Title_Master.,Title_Miss.,Title_Mr.,Title_Mrs.,AgeFill,5_adult,5_aged,5_child,5_senior,Cabin_Category,AgeClass,ClassFare,HighLow
count,418.000000,418.000000,418.000000,418.000000,418.000000,418.000000,418.000000,418.000000,418.000000,418.000000,418.000000,418.000000,418.000000,418.000000,418.000000,418.000000,418.000000,418.000000,418.000000,418.000000,418.000000,418.000000,418.000000,418.000000,418.000000,418.000000
mean,2.265550,0.447368,0.392344,180.944976,35.560845,0.363636,0.636364,0.244019,0.110048,0.645933,1.839713,0.444976,21.770888,0.050239,0.188995,0.583732,0.177033,30.222256,0.461722,0.026316,0.062201,0.449761,0.217703,63.568484,33.040328,0.557416
std,0.841838,0.896760,0.981429,107.533763,55.856972,0.481622,0.481622,0.430019,0.313324,0.478803,1.519072,1.668752,35.603507,0.218700,0.391974,0.493530,0.382154,13.021233,0.499130,0.160265,0.241810,0.498066,0.413179,29.849900,34.798837,0.497288
min,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.170000,0.000000,0.000000,0.000000,0.000000,0.000000,0.510000,0.000000,0.000000
25%,1.000000,0.000000,0.000000,85.250000,7.895800,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,7.634400,0.000000,0.000000,0.000000,0.000000,21.831133,0.000000,0.000000,0.000000,0.000000,0.000000,42.000000,21.675000,0.000000
50%,3.000000,0.000000,0.000000,181.000000,14.454200,0.000000,1.000000,0.000000,0.000000,1.000000,1.000000,0.000000,8.662500,0.000000,0.000000,1.000000,0.000000,30.000000,0.000000,0.000000,0.000000,0.000000,0.000000,61.500000,23.687400,1.000000
75%,3.000000,1.000000,0.000000,279.750000,31.471875,1.000000,1.000000,0.000000,0.000000,1.000000,2.000000,0.000000,25.982813,0.000000,0.000000,1.000000,0.000000,36.875000,1.000000,0.000000,0.000000,1.000000,0.000000,86.750000,27.720800,1.000000
max,3.000000,8.000000,9.000000,362.000000,512.329200,1.000000,1.000000,1.000000,1.000000,1.000000,11.000000,16.000000,262.375000,1.000000,1.000000,1.000000,1.000000,76.000000,1.000000,1.000000,1.000000,1.000000,1.000000,181.500000,262.375000,1.000000


In [29]:
clf = RandomForestClassifier(n_estimators = 500, max_depth = 7)
clf.fit(x, y)
output = clf.predict(test_new)
test_ids = test['PassengerId']

In [30]:
predictions_file = open("titanic_20160704.csv", "wb")
open_file_object = csv.writer(predictions_file)
open_file_object.writerow(["PassengerId","Survived"])
open_file_object.writerows(zip(test_ids, output))
predictions_file.close()

This will result in a prediction accuracy of 78.469%, which is better but not there yet :(

In [86]:
test_new.describe()

,Pclass,SibSp,Parch,Fare,female,male,Embarked_C,Embarked_Q,Embarked_S,FamilySize,Family,Fare_Per_Person,Title_Master.,Title_Miss.,Title_Mr.,Title_Mrs.,AgeFill,AgeCategory_adult,AgeCategory_aged,AgeCategory_child,AgeCategory_senior,Cabin_Category,AgeClass,ClassFare,HighLow
count,418.000000,418.000000,418.000000,418.000000,418.000000,418.000000,418.000000,418.000000,418.000000,418.000000,418.000000,418.000000,418.000000,418.000000,418.000000,418.000000,418.000000,418.000000,418.000000,418.000000,418.000000,418.000000,418.000000,418.000000,418.000000
mean,2.265550,0.447368,0.392344,35.560845,0.363636,0.636364,0.244019,0.110048,0.645933,1.839713,0.444976,21.770888,0.050239,0.188995,0.583732,0.177033,30.222256,0.461722,0.026316,0.062201,0.449761,0.217703,63.568484,33.040328,0.557416
std,0.841838,0.896760,0.981429,55.856972,0.481622,0.481622,0.430019,0.313324,0.478803,1.519072,1.668752,35.603507,0.218700,0.391974,0.493530,0.382154,13.021233,0.499130,0.160265,0.241810,0.498066,0.413179,29.849900,34.798837,0.497288
min,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.170000,0.000000,0.000000,0.000000,0.000000,0.000000,0.510000,0.000000,0.000000
25%,1.000000,0.000000,0.000000,7.895800,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,7.634400,0.000000,0.000000,0.000000,0.000000,21.831133,0.000000,0.000000,0.000000,0.000000,0.000000,42.000000,21.675000,0.000000
50%,3.000000,0.000000,0.000000,14.454200,0.000000,1.000000,0.000000,0.000000,1.000000,1.000000,0.000000,8.662500,0.000000,0.000000,1.000000,0.000000,30.000000,0.000000,0.000000,0.000000,0.000000,0.000000,61.500000,23.687400,1.000000
75%,3.000000,1.000000,0.000000,31.471875,1.000000,1.000000,0.000000,0.000000,1.000000,2.000000,0.000000,25.982813,0.000000,0.000000,1.000000,0.000000,36.875000,1.000000,0.000000,0.000000,1.000000,0.000000,86.750000,27.720800,1.000000
max,3.000000,8.000000,9.000000,512.329200,1.000000,1.000000,1.000000,1.000000,1.000000,11.000000,16.000000,262.375000,1.000000,1.000000,1.000000,1.000000,76.000000,1.000000,1.000000,1.000000,1.000000,1.000000,181.500000,262.375000,1.000000
